# Face swapping local avec .venv (notebook)

Ce notebook est conçu pour être exécuté dans un environnement Python créé avec `python -m venv` (pas conda). Il aide à :
- vérifier que tu es bien dans un .venv
- installer les dépendances (hors torch) dans le .venv
- définir les chemins vers ton dossier `fsgan` cloné et vers les poids (.pth)
- initialiser l'objet `FaceSwapping` (si les poids existent)
- te montrer comment lancer l'inférence ou où trouver le script/méthode d'appel

Avant d'exécuter :
1. Crée et active ton .venv en dehors du notebook (terminal) :
   python3 -m venv ~/venv_fsgan
   source ~/venv_fsgan/bin/activate

2. Installe la bonne version de torch pour ta machine (exemples) :
   - CPU-only :
     python -m pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
   - CUDA 11.7 (si tu as CUDA 11.7) :
     python -m pip install torch torchvision --index-url https://download.pytorch.org/whl/cu117

3. Lance Jupyter depuis le .venv :
   python -m pip install jupyter
   source ~/venv_fsgan/bin/activate
   jupyter notebook

4. Ouvre ce notebook dans Jupyter (toujours avec le .venv activé).

Remarque : adapte les variables REPO_ROOT et WEIGHTS_DIR dans la cellule `Configuration des chemins` plus bas.

In [ ]:
import sys, os
print('Python executable:', sys.executable)
print('Python version:', sys.version)
in_venv = getattr(sys, 'base_prefix', None) != sys.prefix
print('Dans un venv:', in_venv)
print('\nChemins importants:')
print('sys.path[0]:', sys.path[0])


Cellule suivante : installe les dépendances Python (sauf torch). Le notebook utilisera `sys.executable -m pip` pour installer dans le .venv actif.

In [ ]:
import subprocess, sys
reqs = ['opencv-python', 'ffmpeg-python', 'youtube-dl', 'yacs', 'tqdm', 'moviepy']
print('Installation (ou vérification) des dépendances:', reqs)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + reqs)
print('Dépendances installées (hors torch).')

Configuration des chemins. Modifie REPO_ROOT et WEIGHTS_DIR pour pointer vers ton clone `fsgan` et le dossier contenant les .pth.

In [ ]:
# ========== ADAPTE CES CHEMINS AVANT D'EXÉCUTER ========== 
REPO_ROOT = 'fsgan/'        # ex: '/home/ton_user/projects/fsgan'
WEIGHTS_DIR = 'weights/'      # ex: '/home/ton_user/projects/fsgan_weights'
# ======================================================
print('REPO_ROOT =', REPO_ROOT)
print('WEIGHTS_DIR =', WEIGHTS_DIR)
print('\nContenu du dossier inference (si REPO_ROOT correct) :')
try:
    print('\n'.join(os.listdir(os.path.join(REPO_ROOT, 'inference'))))
except Exception as e:
    print('Impossible de lister inference/:', e)


Initialisation de fsgan : ajout du repo au PYTHONPATH, import des modules et vérification des poids. Si l'import de torch échoue, assure-toi d'avoir installé torch dans ton .venv (voir instructions plus haut).

In [ ]:
import sys, os
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

missing = []
expected = [
    os.path.join(WEIGHTS_DIR, 'WIDERFace_DSFD_RES152.pth'),
    os.path.join(WEIGHTS_DIR, 'hopenet_robust_alpha1.pth'),
    os.path.join(WEIGHTS_DIR, 'hr18_wflw_landmarks.pth'),
    os.path.join(WEIGHTS_DIR, 'celeba_unet_256_1_2_segmentation_v2.pth'),
    os.path.join(WEIGHTS_DIR, 'nfv_msrunet_256_1_2_reenactment_v2.1.pth'),
    os.path.join(WEIGHTS_DIR, 'ijbc_msrunet_256_1_2_inpainting_v2.pth'),
    os.path.join(WEIGHTS_DIR, 'ijbc_msrunet_256_1_2_blending_v2.pth'),
    os.path.join(WEIGHTS_DIR, 'vggface2_vgg19_256_1_2_id.pth'),
]
for p in expected:
    if not os.path.exists(p):
        missing.append(p)

if missing:
    print('ATTENTION : fichiers de poids manquants :')
    for m in missing:
        print(' -', m)
    print('\nPlace les fichiers .pth requis dans WEIGHTS_DIR ou adapte la liste si tu possèdes d autres fichiers.')
else:
    print('Tous les fichiers de poids attendus semblent présents (vérifie les noms exacts si besoin).')

try:
    import torch
    print('Torch importé :', torch.__version__)
except Exception as e:
    print('Impossible d\'importer torch. Installe torch dans ton .venv avant de continuer. Erreur:', e)

try:
    from fsgan.inference.swap import FaceSwapping
    from fsgan.criterions.vgg_loss import VGGLoss
    print('\nImport fsgan réussi.')
except Exception as e:
    print('\nImpossible d\'importer fsgan depuis REPO_ROOT. Vérifie que REPO_ROOT est correct et que le dossier contient un package fsgan. Erreur:', e)


Initialisation de l'objet FaceSwapping (exemple). Si une erreur survient, lis le message et ajuste les chemins/versions. La cellule créée ci-dessous correspond aux noms de poids standards du dépôt — adapte si nécessaire.

In [ ]:
try:
    import torch
    # Force tous les chargements de tensors sur CPU
    torch.set_default_device('cpu')
    device = torch.device('cpu')
    
    # Définit les chemins aux modèles
    detection_model = os.path.join(WEIGHTS_DIR, 'WIDERFace_DSFD_RES152.pth')
    pose_model = os.path.join(WEIGHTS_DIR, 'hopenet_robust_alpha1.pth')
    lms_model = os.path.join(WEIGHTS_DIR, 'hr18_wflw_landmarks.pth')
    seg_model = os.path.join(WEIGHTS_DIR, 'celeba_unet_256_1_2_segmentation_v2.pth')
    reenactment_model = os.path.join(WEIGHTS_DIR, 'nfv_msrunet_256_1_2_reenactment_v2.1.pth')
    completion_model = os.path.join(WEIGHTS_DIR, 'ijbc_msrunet_256_1_2_inpainting_v2.pth')
    blending_model = os.path.join(WEIGHTS_DIR, 'ijbc_msrunet_256_1_2_blending_v2.pth')
    criterion_id_path = os.path.join(WEIGHTS_DIR, 'vggface2_vgg19_256_1_2_id.pth')

    # Patch temporaire pour forcer le chargement sur CPU
    original_load = torch.load
    torch.load = lambda *args, **kwargs: original_load(*args, **{**kwargs, 'map_location': 'cpu'})
    
    criterion_id = VGGLoss(criterion_id_path)
    
    # Restaure torch.load
    torch.load = original_load

    print('VGGLoss initialisé avec succès sur CPU.')
except Exception as e:
    print('Erreur lors de l\'initialisation :', e)
    import traceback
    traceback.print_exc()

Trouver et appeler la méthode d'inférence :
- Le dépôt fournit parfois un script CLI dans `inference/` et/ou une méthode sur l'objet FaceSwapping.
- La cellule suivante liste les fichiers dans `inference/` pour t'aider à repérer un script exécutable (par ex `face_swap.py` ou `swap.py`).

In [ ]:
inference_dir = os.path.join(REPO_ROOT, 'inference')
if os.path.isdir(inference_dir):
    files = os.listdir(inference_dir)
    print('Fichiers dans inference/:')
    for f in files:
        print(' -', f)
    # Si existe, affiche le début d'un fichier swap.py pour retrouver la fonction
    candidate = os.path.join(inference_dir, 'swap.py')
    candidate2 = os.path.join(inference_dir, 'face_swap.py')
    for c in (candidate, candidate2):
        if os.path.exists(c):
            print('\n--- Début de', os.path.basename(c), '---')
            with open(c, 'r', encoding='utf-8', errors='ignore') as fh:
                for i, line in enumerate(fh):
                    print(line.rstrip())
                    if i > 200:
                        break
            break
else:
    print('Dossier inference/ introuvable : vérifie REPO_ROOT.')


Exemple d'utilisation si le dépôt propose un script CLI (exécute dans le .venv, cell suivante exécute le script via subprocess) :
- Adapte `script_name` et les arguments selon le script présent.
- Si tu préfères appeler une méthode Python sur `face_swapping`, ouvre `fsgan/inference/swap.py` pour connaître l'API et adapte l'appel.

In [ ]:
# Chemins des images'C:\\dev\\ProjetImage\\Face-Swap-M2\\data\\kirk.jpg'
source_path = 'C:\\dev\\ProjetImage\\Face-Swap-M2\\data\\Face-Swap-M2-Dataset\\dataset\\CelebAex\\3\\3.jpg'
target_path = 'C:\\dev\\ProjetImage\\Face-Swap-M2\\data\\Face-Swap-M2-Dataset\\dataset\\CelebAex\\1\\1.jpg'
output_path = 'C:\\dev\\ProjetImage\\Face-Swap-M2\\data\\out_swapped.jpg'

# Cellule Coeur qui execute le swap


In [ ]:
import subprocess
import os
import cv2

script_name = os.path.join(REPO_ROOT, 'inference', 'swap.py')
source_video = os.path.expanduser(source_path)
target_video = os.path.expanduser(target_path)
output_video = os.path.expanduser('C:\\dev\\ProjetImage\\Face-Swap-M2\\data\\out_swapped_temp.mp4')
output_image = os.path.expanduser(output_path)

if os.path.exists(script_name):
    env = os.environ.copy()
    repo_root_abs = os.path.abspath('.')
    if 'PYTHONPATH' in env:
        env['PYTHONPATH'] = repo_root_abs + os.pathsep + env['PYTHONPATH']
    else:
        env['PYTHONPATH'] = repo_root_abs
    
    env['PYTHONIOENCODING'] = 'utf-8'
    
    # Ajouter FFmpeg au PATH si installé localement
    ffmpeg_path = 'C:\\ffmpeg\\bin'
    if os.path.exists(ffmpeg_path):
        env['PATH'] = ffmpeg_path + os.pathsep + env.get('PATH', '')
    
    cmd = [sys.executable, script_name, source_video, '-t', target_video, '-o', output_video, 
        '--cpu_only',           # CPU uniquement
        '--output_crop',        # Sortie croppée (important!)
        '--finetune',           # Fine-tuning pour meilleure qualité
        '--finetune_iterations', '600',  # Nombre d'itérations
        '--resolution', '256',  # Résolution (augmentez si bonne machine)
        '--crop_scale', '1.2',  # Échelle du crop
        '--batch_size', '4',    # Batch size réduit pour CPU
        '--min_radius', '1.5',  # Densité appearance map
        '--verbose', '0']       # Pas de visualisation debug
    
    print('Exécution de la commande:')
    print(' '.join(cmd))
    print(f'PYTHONPATH: {env["PYTHONPATH"]}')
    print('\n--- Début de l\'exécution ---')
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, env=env, encoding='utf-8', errors='replace')
        print(result.stdout)
        if result.returncode != 0:
            print('ERREUR:')
            print(result.stderr)
        else:
            print('\n--- Fin de l\'exécution ---')
            print(f'Vidéo temporaire sauvegardée dans: {output_video}')
            
            # Extraire la première frame et la sauvegarder en JPG
            if os.path.exists(output_video):
                cap = cv2.VideoCapture(output_video)
                ret, frame = cap.read()
                cap.release()
                
                if ret:
                    cv2.imwrite(output_image, frame)
                    print(f'Image extraite et sauvegardée dans: {output_image}')
                    
                    # Supprimer la vidéo temporaire
                    os.remove(output_video)
                    print(f'Vidéo temporaire supprimée')
                else:
                    print('Erreur lors de l\'extraction de la frame')
    except Exception as e:
        print(f'Exception: {e}')
        import traceback
        traceback.print_exc()
else:
    print('Script CLI', script_name, 'introuvable.')

Fonction d'aide pour afficher une vidéo résultat dans le notebook (utilise IPython.display.Video).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

# Charger les images
source_img = cv2.imread(source_path)
target_img = cv2.imread(target_path)
output_img = cv2.imread(output_path)

# Vérifier que les images sont chargées
if source_img is None:
    print(f"Erreur: impossible de charger {source_path}")
if target_img is None:
    print(f"Erreur: impossible de charger {target_path}")
if output_img is None:
    print(f"Erreur: impossible de charger {output_path}")

# Convertir BGR en RGB pour matplotlib
source_img = cv2.cvtColor(source_img, cv2.COLOR_BGR2RGB)
target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)
output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)

# Créer une figure avec 3 sous-graphiques
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Afficher les images
axes[0].imshow(source_img)
axes[0].set_title('Source (Andrew)', fontsize=14)
axes[0].axis('off')

axes[1].imshow(target_img)
axes[1].set_title('Target (Arthur)', fontsize=14)
axes[1].axis('off')

axes[2].imshow(output_img)
axes[2].set_title('Output (Swapped)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.show()